# Langchain Retrieval QA 文心一言例子

参考：https://python.langchain.com/docs/modules/chains/popular/vector_db_qa

并使用中文文本：《拿来主义》进行提问，同时和 OpenAI 效果进行对比。

In [ ]:
!pip install langchain langchain-wenxin openai chromadb tiktoken

In [30]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAIChat
from langchain_wenxin.llms import Wenxin
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.model_laboratory import ModelLaboratory

In [15]:
import getpass
baidu_api_key = getpass.getpass("Baidu API Key:")
baidu_secret_key = getpass.getpass("Baidu Secret Key:")

Baidu API Key:··········
Baidu Secret Key:··········


In [20]:
openai_api_key = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [18]:
text = """拿来主义


中国一向是所谓“闭关主义”，自己不去，别人也不许来。自从给枪炮打破了大门之后，又碰了一串钉子，到现在，成了什么都是“送去主义”了。别的且不说罢，单是学艺上的东西，近来就先送一批古董到巴黎去展览，但终“不知后事如何”；还有几位“大师”们捧着几张古画和新画，在欧洲各国一路的挂过去，叫作“发扬国光”〔2〕。听说不远还要送梅兰芳博士到苏联去，以催进“象征主义”〔3〕，此后是顺便到欧洲传道。我在这里不想讨论梅博士演艺和象征主义的关系，总之，活人替代了古董，我敢说，也可以算得显出一点进步了。

但我们没有人根据了“礼尚往来”的仪节，说道：拿来！

当然，能够只是送出去，也不算坏事情，一者见得丰富，二者见得大度。尼采〔4〕就自诩过他是太阳，光热无穷，只是给与，不想取得。然而尼采究竟不是太阳，他发了疯。中国也不是，虽然有人说，掘起地下的煤来，就足够全世界几百年之用，但是，几百年之后呢？几百年之后，我们当然是化为魂灵，或上天堂，或落了地狱，但我们的子孙是在的，所以还应该给他们留下一点礼品。要不然，则当佳节大典之际，他们拿不出东西来，只好磕头贺喜，讨一点残羹冷炙做奖赏。这种奖赏，不要误解为“抛来”的东西，这是“抛给”的，说得冠冕些，可以称之为“送来”，我在这里不想举出实例〔5〕。

我在这里也并不想对于“送去”再说什么，否则太不“摩登”了。我只想鼓吹我们再吝啬一点，“送去”之外，还得“拿来”，是为“拿来主义”。

但我们被“送来”的东西吓怕了。先有英国的鸦片，德国的废枪炮，后有法国的香粉，美国的电影，日本的印着“完全国货”的各种小东西。于是连清醒的青年们，也对于洋货发生了恐怖。其实，这正是因为那是“送来”的，而不是“拿来”的缘故。

所以我们要运用脑髓，放出眼光，自己来拿！

譬如罢，我们之中的一个穷青年，因为祖上的阴功（姑且让我这么说说罢），得了一所大宅子，且不问他是骗来的，抢来的，或合法继承的，或是做了女婿换来的。那么，怎么办呢？我想，首先是不管三七二十一，“拿来”！但是，如果反对这宅子的旧主人，怕给他的东西染污了，徘徊不敢走进门，是孱头；勃然大怒，放一把火烧光，算是保存自己的清白，则是昏蛋。不过因为原是羡慕这宅子的旧主人的，而这回接受一切，欣欣然的蹩进卧室，大吸剩下的鸦片，那当然更是废物。“拿来主义”者是全不这样的。

他占有，挑选。看见鱼翅，并不就抛在路上以显其“平民化”，只要有养料，也和朋友们像萝卜白菜一样的吃掉，只不用它来宴大宾；看见鸦片，也不当众摔在毛厕里，以见其彻底革命，只送到药房里去，以供治病之用，却不弄“出售存膏，售完即止”的玄虚。只有烟枪和烟灯，虽然形式和印度，波斯，阿剌伯的烟具都不同，确可以算是一种国粹，倘使背着周游世界，一定会有人看，但我想，除了送一点进博物馆之外，其余的是大可以毁掉的了。还有一群姨太太，也大以请她们各自走散为是，要不然，“拿来主义”怕未免有些危机。

总之，我们要拿来。我们要或使用，或存放，或毁灭。那么，主人是新主人，宅子也就会成为新宅子。然而首先要这人沉着，勇猛，有辨别，不自私。没有拿来的，人不能自成为新人，没有拿来的，文艺不能自成为新文艺。

六月四日。



〔1〕本篇最初发表于一九三四年六月七日《中华日报·动向》，署名霍冲。

〔2〕“发扬国光”一九三二年至一九三四年间，美术家徐悲鸿、刘海粟曾分别去欧洲一些国家举办中国美术展览或个人美术作品展览。“发扬国光”是一九三四年五月二十八日《大晚报》报道这些消息时的用语。

〔3〕“象征主义”一九三四年五月二十八日《大晚报》报道：“苏俄艺术界向分写实与象征两派，现写实主义已渐没落，而象征主义则经朝野一致提倡，引成欣欣向荣之概。自彼邦艺术家见我国之书画作品深合象征派后，即忆及中国戏剧亦必采取象征主义。因拟……邀中国戏曲名家梅兰芳等前往奏艺。”鲁迅曾在《花边文学·谁在没落》一文中批评《大晚报》的这种歪曲报道。

〔4〕尼采（ＦＮｉｅｔｚｓｃｈｅ，1844—1900）德国哲学家，唯意志论和“超人”哲学的鼓吹者。这里所述尼采的话，见于他的《札拉图斯特拉如是说·序言》。

〔5〕一九三三年六月四日，国民党政府和美国在华盛顿签订五千万美元的“棉麦借款”，购买美国的小麦、面粉和棉花。这里指的可能是这一类事。
"""

with open("data.txt", "w") as f:
  f.write(text)

In [34]:
loader = TextLoader("data.txt")
documents = loader.load()
separators = ["\n\n", "\n", "。", "？", "！", "；", " ", ""]
text_splitter = RecursiveCharacterTextSplitter(separators=separators, chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
docsearch = Chroma.from_documents(texts, embeddings)

In [39]:
from langchain.prompts import PromptTemplate
prompt_template = """使用以下上下文回答最后的问题。如果在上下文中没有找到答案，请直接说「答案不在上下文中」，不要试图编造一个答案。

{context}

问题：{question}
中文回答："""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

llms = [
    OpenAIChat(model="gpt-3.5-turbo", openai_api_key=openai_api_key, temperature=0),
    Wenxin(model="ernie-bot", baidu_api_key=baidu_api_key, baidu_secret_key=baidu_secret_key, temperature=0.1),
]


chains = [
    RetrievalQA.from_chain_type(llms[0], chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs),
    RetrievalQA.from_chain_type(llms[1], chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs),
]

names = [
    str(llms[0]),
    str(llms[1]),
]

model_lab = ModelLaboratory(chains, names=names)

In [41]:
# I know it.
model_lab.compare("拿来主义者碰到鱼翅怎么办？")

# I don't know.
model_lab.compare("什么是犬儒主义？")

Input:
拿来主义者碰到鱼翅怎么办？

OpenAIChat
Params: {'model_name': 'gpt-3.5-turbo', 'model': 'gpt-3.5-turbo', 'temperature': 0}
拿来主义者会吃掉鱼翅，但不会用它来宴请大宾。

Wenxin
Params: {}
拿来主义者应该吃掉鱼翅，只要养料足够，就和朋友们像萝卜白菜一样的吃掉。

Input:
什么是犬儒主义？

OpenAIChat
Params: {'model_name': 'gpt-3.5-turbo', 'model': 'gpt-3.5-turbo', 'temperature': 0}
答案不在上下文中。

Wenxin
Params: {}
犬儒主义（Cynicism）是一种哲学观点，认为犬儒主义是一种社会哲学，旨在通过消除人类文明社会的价值观和道德标准来重建社会秩序。犬儒主义认为，人类社会存在着一种普遍的堕落和虚伪，使得人们无法真正地追求真理和正义，因此必须采取一种反传统的、怀疑和批判的态度来对待社会和道德问题。犬儒主义通常与道德相对立，认为道德是一种虚伪和欺骗，只有通过否定道德和价值观才能真正地追求真理和正义。

犬儒主义是一种极端的思想，通常与主流社会价值观和道德标准相悖。它通常被认为是一种消极的、反社会的思想，因为它否认了人类社会的价值和道德标准，并且认为人们应该采取一种消极、自我放纵的态度来对待生活。这种思想在历史上曾经产生过一些负面影响，例如导致了社会的混乱和道德标准的降低。

因此，我们应该采取一种积极的态度来对待社会和道德问题，尊重主流价值观和道德标准，并且努力推动社会的进步和发展。

